In [2]:
import loadexcel as lc
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
from data_convert import*
from tqdm import tqdm
import feedback_option
import loaddb as ld
import os

class DataManager():
    
    def __init__(self,date,path):
        self._date=date
        self._path=path
        self.update_p_index()
        self._car_value=ld.get_car_info()
    def getPath(self):
        return self._path
    
    def get_data(self):
        return self._data
    
    def update_path(self,new_path):
        self._path=new_path
        self.update_p_index()
        
    def update_p_index(self):
        self.p_index=len(os.path.split(self._path)[0].split("//"))+1
        
    def load_path(self,path):
        self._path=path
        self.update_p_index()
        self.load()
    
    def get_Sum(self):
        return self._clean_sum
    
    def get_detail(self):
        return self._clean_detail
        
    def load(self):
        result_=lc.excel2Dataframe(self._path,target_by="车号",date_=self._date)
        sum_,detail_=[],[]
        self._pd_sum=[]
        self._pd_detail=[]
        for item in result_:
            sum_+=item[0]
            detail_+=item[1]
        for item in tqdm(sum_):
            self._pd_sum.append(lc.excel2Dataframe_Sum(item["filepath"],self.p_index,item["sheetname"],"车号"))
        for item in tqdm(detail_):
            self._pd_detail.append(lc.excel2Dataframe_Detail(item["filepath"],self.p_index,item["sheetname"],"车号"))
        self._pd_sum=pd.concat(self._pd_sum)
        self._pd_detail=pd.concat(self._pd_detail)
        self._clean_sum=self.clean_data(self._pd_sum)
        self._clean_detail=self._pd_detail
    #获取不匹配的数据    
    def get_dismatchs(df_source):
        #连接数据与车辆信息
        result_data=pd.merge(df_source,self._car_value,left_on='car_id',right_on='sub_car_id',how='left')
        #找出无法匹配的车辆
        outer_result=result_data[pd.isnull(result_data['sub_car_id'])]
        dismacths=outer_result[["route_x","team","car_id_x"]]
        return dismacths
    #过滤一些系统能够识别的数据
    def filter(df_source,dismacths):
        for i in range(len(dismacths)) :
            _car=dismacths.iloc[i]
            car_id_=_car["car_id_x"]
            route_=_car["route_x"]
            _r=self._car_value[self._car_value["route"]==route_]
            result_r=_r[_r["sub_car_id"].str.contains(car_id_)]
            if len(result_r)==1:
                dismacths.iloc[i]["car_id_x"]=result_r["sub_car_id"].iloc[0]
        df_source.iloc[un_turn_data.index,0]=un_turn_data['car_id_x']
        return df_source
        
    #清理
    def clean_data(df_source):
        dismacths=get_dismatchs(df_source)
        filter_data=filter(df_source,dismacths)
        dismacths=get_dismatchs(df_source)
        if len(dismacths)>0:
            #------------TODO——————抛出异常
            return 1
        if len(dismatchs)==0:
            return 0
            
        
        
        

In [3]:
import datetime

In [4]:
dm=DataManager(datetime.date(2019,8,1),"example_data//feedback//8月原始")

In [5]:
dm.load_path("example_data//feedback//8月原始")

100%|██████████████████████████████████████████████████████████████████████████████████| 58/58 [00:06<00:00,  8.33it/s]


In [6]:
result=dm.get_detail()

In [19]:

#去掉路
remove_route=lambda x:x.split("路")[-1].strip()
#去掉线
remove_line=lambda x:x.split("线")[-1].strip()
#去掉斜杆
remove_diagonal=lambda x:x.split("/")[-1].strip()
#将F改成正常拼写
replaceF=lambda x:x.replace("Ｆ","F")
#如果以F和D结尾，则补齐6位
zfill_ = lambda x: x.zfill(6) if x.endswith("F")|x.endswith("D") else x

In [8]:
result.iloc[:,0]=result.iloc[:,0].apply(str)
result.iloc[:,0]=result.iloc[:,0].apply(remove_route)
result.iloc[:,0]=result.iloc[:,0].apply(remove_line)
result.iloc[:,0]=result.iloc[:,0].apply(remove_diagonal)


In [12]:
result.index=range(len(result))

In [20]:
#将所选列转为字符
result.iloc[:,0]=result.iloc[:,0].apply(str)
#将所选列转为剔除空格
result.iloc[:,0]=result.iloc[:,0].str.strip()

result["car_id"]=result.iloc[:,0].apply(replaceF)
result["car_id"]=result.iloc[:,0].apply(zfill_)

In [24]:
import loaddb as ld
car_value=ld.get_car_info()
#连接油耗数据与指标
result_data=pd.merge(result,car_value,left_on='car_id',right_on='sub_car_id',how='left')
outer_result=result_data[pd.isnull(result_data['sub_car_id'])]


In [54]:
un_turn_data=outer_result[["route_x","team","car_id_x"]]

In [56]:
for i in range(len(un_turn_data)) :
    _car=un_turn_data.iloc[i]
    car_id_=_car["car_id_x"]
    route_=_car["route_x"]
    _r=car_value[car_value["route"]==route_]
    result_r=_r[_r["sub_car_id"].str.contains(car_id_)]
    if len(result_r)==1:
        un_turn_data.iloc[i]["car_id_x"]=result_r["sub_car_id"].iloc[0]
un_turn_data

,route_x,team,car_id_x
381,199,4,01866F
382,199,4,02858F
383,199,4,02896F
384,199,4,03222F
385,199,4,03288F
386,199,4,05515F
387,199,4,05522F
388,199,4,05525F
389,199,4,05558F
390,199,4,05566F


In [ ]:
data.iloc[un_turn_data.index,0]=un_turn_data['car_id_x']

In [53]:
if len(outer_result) >0:
    for i in range(len(outer_result)):
        item=outer_result.iloc[i][{0,9}]
        print(item['route_x']+"线路下的"+item['car_id_x']+"无法识别")
    print(f"合计%d个项无法识别，请修正后重试"%(len(outer_result)))

199线路下的1866无法识别
199线路下的2858无法识别
199线路下的2896无法识别
199线路下的3222无法识别
199线路下的3288无法识别
199线路下的5515无法识别
199线路下的5522无法识别
199线路下的5525无法识别
199线路下的5558无法识别
199线路下的5566无法识别
199线路下的5599无法识别
35线路下的600无法识别
35线路下的5226无法识别
35线路下的6000无法识别
35线路下的6068无法识别
35线路下的6088无法识别
35线路下的8118无法识别
35线路下的8199无法识别
35线路下的8859无法识别
夜班一号线线路下的71无法识别
夜班一号线线路下的113无法识别
夜班一号线线路下的211无法识别
149线路下的01189无法识别
83线路下的303无法识别
83线路下的382无法识别
83线路下的2018无法识别
83线路下的2100无法识别
83线路下的3127无法识别
83线路下的3189无法识别
83线路下的3230无法识别
83线路下的3296无法识别
83线路下的3883无法识别
83线路下的5708无法识别
83线路下的5738无法识别
83线路下的6213无法识别
83线路下的6329无法识别
83线路下的7100无法识别
83线路下的7168无法识别
83线路下的8602无法识别
83线路下的8613无法识别
83线路下的8629无法识别
83线路下的9239无法识别
83线路下的9612无法识别
83线路下的603无法识别
合计44个项无法识别，请修正后重试
